In [4]:
s = """
Titanic Machine learning from disaster

Columns     DESCRIPTION:
survival	Survival	            0 = No, 1 = Yes
pclass      Ticket class	        1 = 1st, 2 = 2nd, 3 = 3rd
sex	        Sex	
Age	        Age in years	
sibsp	    # of siblings / spouses aboard the Titanic	
parch	    # of parents / children aboard the Titanic	
ticket	    Ticket number	
fare	    Passenger fare	
cabin	    Cabin number	
embarked	Port of Embarkation	    C = Cherbourg, Q = Queenstown, S = Southampton

"""
print(s)


Titanic Machine learning from disaster

Columns     DESCRIPTION:
survival	Survival	            0 = No, 1 = Yes
pclass      Ticket class	        1 = 1st, 2 = 2nd, 3 = 3rd
sex	        Sex	
Age	        Age in years	
sibsp	    # of siblings / spouses aboard the Titanic	
parch	    # of parents / children aboard the Titanic	
ticket	    Ticket number	
fare	    Passenger fare	
cabin	    Cabin number	
embarked	Port of Embarkation	    C = Cherbourg, Q = Queenstown, S = Southampton




In [24]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from  sklearn.ensemble import RandomForestClassifier
from tensorflow import keras
warnings.filterwarnings(action = 'ignore')

In [6]:
PATH = "..\Titanic - Machine learning from disaster\\titanic"
    
FILES = os.listdir(PATH)

xTrain = pd.read_csv(PATH + "\\" + "train.csv")

xTest = pd.read_csv(PATH + "\\" + "test.csv")

_X = xTrain.drop(columns = 'Survived', inplace = False)
y = xTrain['Survived']

X = pd.concat([_X, xTest], axis = 0)

X.shape

#FILES

(1309, 11)

In [7]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
s = """
Useless features:
    1. passengerID  : random numbers
    2. Cabin number : lot's of NaN value
    3. Ticket number
Feature engineering :
    1. Name : extract titles Mr. Mrs. Dr. 
    2. SipSb Parch : family size
Others:
    1. Pclass : categorical
    2. Sex : categorical
    3. Fare : continuous
    4. Embarked : categorical 
    5. Age : continuous
"""

print(s)


Useless features:
    1. passengerID  : random numbers
    2. Cabin number : lot's of NaN value
    3. Ticket number
Feature engineering :
    1. Name : extract titles Mr. Mrs. Dr. 
    2. SipSb Parch : family size
Others:
    1. Pclass : categorical
    2. Sex : categorical
    3. Fare : continuous
    4. Embarked : categorical 
    5. Age : continuous



In [10]:
useless = ['PassengerId', 'Cabin', 'Ticket']
forFeatEx = ['Name', 'SibSp', 'Parch']
rest = ['Pclass', 'Sex', 'Fare', 'Embarked', 'Age']


In [65]:
# remove useless features
_x = X.drop(columns = useless)
#_xTest = xTest.drop(columns = useless)
_x.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [66]:
# feature extration with name
def featEx_name(df):
    trainL = []
    for _strain in list(df.Name):
        _s = _strain.split(", ")[1]
        trainL.append(_s.split(" ")[0])
    df['Salutation'] = trainL
    return df
def featEx_familySize(df):
    parch = list(df.Parch)
    sibsb = list(df.SibSp)
    familySize = [parch[i] + sibsb[i] for i in range(len(parch))]
    df['FamSize'] = familySize
    return df

In [85]:
__x = featEx_name(_x)
#__xTest = featEx_name(_xTest)
__x.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,FamSize
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,Mr.,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,Mrs.,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,Miss.,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,Mrs.,1
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,Mr.,0


In [86]:
__x = featEx_familySize(__x)
#__xTest = featEx_familySize(__xTest)
__x.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,FamSize
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,Mr.,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,Mrs.,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,Miss.,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,Mrs.,1
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,Mr.,0


In [106]:
#distribution function for age


#temp = __x[__x['Salutation'].value_counts() < 0.01*891]
L = list(set(__x['Salutation']))
misc = []
rareSalutation = list((__x['Salutation'].value_counts() < 0.01*891))
for i in range(len(rareSalutation)):
    if rareSalutation[i] == True:
        misc.append(L[i])
        L.remove(L[i])
print(misc)

IndexError: list index out of range

In [76]:
def createDumm(df, cols):
    for c in cols:
        d = pd.get_dummies(df[c])
        df = pd.concat([df, d], axis = 1)
    return df

categorical = ['Pclass', 'Sex', 'Embarked', 'Salutation']
__x= createDumm(__x, cols = categorical)
#__xTest= createDumm(__xTest, cols = categorical)
__x = __x.drop(columns = categorical, inplace = False)
#__xTest = __xTest.drop(columns = categorical, inplace = False)

__x.head()

,Age,SibSp,Parch,Fare,FamSize,1,2,3,female,male,...,Master.,Miss.,Mlle.,Mme.,Mr.,Mrs.,Ms.,Rev.,Sir.,the
0,22.0,1,0,7.2500,1,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,38.0,1,0,71.2833,1,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,26.0,0,0,7.9250,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,35.0,1,0,53.1000,1,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,35.0,0,0,8.0500,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0


In [77]:
__x = __x.drop(columns = ['Name'])
#__xTest = __xTest.drop(columns = ['Name'])
#__xTest.head()
__x.head()

KeyError: "['Name'] not found in axis"

In [78]:
# lots of NaN values in TRAIN::age, TEST::age, and 1 NaN value in TEST::fare
#__x.fillna(__x.mean(), inplace = True)

#


#__x = imput(__x, mode = 'mean')
__x.isna().sum()

Age          263
SibSp          0
Parch          0
Fare           1
FamSize        0
1              0
2              0
3              0
female         0
male           0
C              0
Q              0
S              0
Capt.          0
Col.           0
Don.           0
Dona.          0
Dr.            0
Jonkheer.      0
Lady.          0
Major.         0
Master.        0
Miss.          0
Mlle.          0
Mme.           0
Mr.            0
Mrs.           0
Ms.            0
Rev.           0
Sir.           0
the            0
dtype: int64

In [79]:
impAge, imputtest= Linreg(__x)

for i in range(len(impAge)):
    if impAge[i] < 0:
        impAge[i] = 0
imputtest['Age'] = list(impAge)

imputtest.head()

,SibSp,Parch,Fare,FamSize,1,2,3,female,male,C,...,Miss.,Mlle.,Mme.,Mr.,Mrs.,Ms.,Rev.,Sir.,the,Age
5,0,0,8.4583,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,34.767479
17,0,0,13.0000,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,32.299339
19,0,0,7.2250,0,0,0,1,1,0,1,...,0,0,0,0,1,0,0,0,0,30.101250
26,0,0,7.2250,0,0,0,1,0,1,1,...,0,0,0,1,0,0,0,0,0,26.960442
28,0,0,7.8792,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,23.693428


In [80]:
for i in imputtest.index:
    __x.at[i, 'Age'] = imputtest.loc[i].Age
    
__x.fillna(0, inplace = True)

In [82]:
#sns.distplot(__x['Age'], bins = 5)
__x['Agebin'] = pd.cut(__x['Age'].astype(int), 5)
__x.head()

,Age,SibSp,Parch,Fare,FamSize,1,2,3,female,male,...,Miss.,Mlle.,Mme.,Mr.,Mrs.,Ms.,Rev.,Sir.,the,Agebin
0,22.0,1,0,7.2500,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,"(16.0, 32.0]"
1,38.0,1,0,71.2833,1,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,"(32.0, 48.0]"
2,26.0,0,0,7.9250,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,"(16.0, 32.0]"
3,35.0,1,0,53.1000,1,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,"(32.0, 48.0]"
4,35.0,0,0,8.0500,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,"(32.0, 48.0]"


KeyError: 'Salutation'

In [61]:
def useLR(X, y):
    _lenofy = y.shape[0]
    xtrain = X[0:_lenofy]
    xtest = X[_lenofy:]
    
    lr = LogisticRegression()
    
    lr.fit(xtrain, y)
    
    return lr.predict(xtest)

def Linreg ( __x):

   
    imputtest = __x[ __x['Age'].isna() ].drop(columns = 'Age', inplace = False)
    
    imputtrain = __x[ __x['Age'] >= 0]
    
    imputtrainx = imputtrain.drop(columns = 'Age', inplace = False)
    imputtrainy = imputtrain['Age']
    
    imputtrainx.fillna(0, inplace = True)
    imputtrainx.fillna(0, inplace = True)
    imputtest.fillna(0, inplace = True)
    #print(imputtrainx.isna().any(), imputtest.isna().any())

    linr = LinearRegression()
    
    linr.fit(imputtrainx, imputtrainy)
    
    return list(linr.predict(imputtest)), imputtest
    

In [ ]:
res = useLR(__x, y)
res
data = {'PassengerId' : list(range(892, 1310)), 'Survived' : res }

df = pd.DataFrame(data)
df.to_csv("result.csv")

In [26]:
def useAnn(X, y):
    _lenofy = y.shape[0]
    xtrain = X[0:_lenofy]
    xtest = X[_lenofy:]
    
    model = keras.Sequential([
        keras.layers.Dense(60, activation = "relu"),
        keras.layers.Dense(1)
    ]
    )
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.fit(xtrain, [0, 1], epochs = 10)
    #test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
    predictions = probability_model.predict(xtest)
    return predictions


Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')